# Toward an Engineering Notebook for Systems Modeling

## *Ed Seidewitz, Model Driven Solutions, Chief Technology Officer*

&copy; Copyright 2019, 2023 Model Driven Solutions, Inc.

# Systems Modeling Language™ (SysML<sup>®</sup>)
Supports the specification, analysis, design, and verification and validation of complex systems that may include hardware, software, information, processes, personnel, and facilities

### SysML v1.0 adopted in 2006
Adopted by the Object Management Group (OMG) as a standard profile of UML v2.1

### SysML v1.7 adopted in 2022
(https://www.omg.org/spec/SYSML/1.7/Beta1)

v1.7 is expected to be the last for SysML v1

### SysML v2.0 Beta adopted in 2023
(https://www.omg.org/spec/SYSML/2.0/Beta1)

Finalization is expected to be complete in 2024.

## SysML v2 Pilot Implementation

### Developed incrementally
The pilot implementation was developed incrementally by the SysML v2 Submission Team (SST) while working on the language specification.

### Open source
Available at https://github.com/Systems-Modeling

### Implements editing of textual notation, with some basic graphical visualization
SysML v2 has a complete textual as well as graphical notation

But the textual notation can be implemented more quickly and incrementally than graphical notation

# Jupyter

## Project Jupyter

"Project Jupyter exists to develop open-source software, open-standards, and services for interactive computing" (http://jupyter.org)

### Interactive programming
Originally based on the Python programming language

Now supports "dozens of programming languages"

### Jupyter Notebook
"The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text." 

## Jupyter and SysML v2
The SysML v2 textual notation is integrated as another Jupyter "language kernel"

### Goals

*Near term*
* A way to quickly create SysML v2 models
* A "no install" platform to track the latest incremental releases of the language 
  * via the JupyterHub deployment available to SST members at http://jupyter.openmbee.org

*Long term*
* A model-based engineering analysis environment
* A means to interchange and integrate system engineering models

# Mass Rollup Demo

## Massed Things

To create a simple mass rollup example, we first define the general concept of a `MassedThing` with two mass attributes:
* `mass` - The mass of the thing itself, not including the mass of any composite parts it may have.
* `totalMass` - The total mass of the thing, including the mass of its composite parts.

In [2]:
part def MassedThing {
    attribute mass :> ISQ::mass; 
    attribute totalMass :> ISQ::mass;
}

PartDefinition MassedThing (93ff128e-c180-4a15-a101-c5f72829652e)


## Simple and Composite Things

Next we define two kinds of usages of `MassedThing`:
* `simpleThing`, which has no composite parts, so its `totalMass` is just the same as its `mass`.
* `compositeThing`, which has composite parts (`subcomponents`), so its `totalMass` is the some of its `mass` plus the `totalMass` of all its composite parts.

In [7]:
part simpleThing : MassedThing {
    attribute redefines totalMass = mass;
}

part compositeThing : MassedThing {
    import NumericalFunctions::sum; 
    
    part subcomponents: MassedThing[*];

    attribute redefines totalMass =
        mass + sum(subcomponents.totalMass); 
}

PartUsage simpleThing (818ca0a2-ed47-48c4-b042-b6c6591c1d1d)
PartUsage compositeThing (8c111cf0-bf98-4794-8e96-a1d881a89f09)


## Spacecraft Components and Systems

Then we can apply the general model to specify that a `SpacecraftComponent` is a `MassedThing` and a `spacecraftSystem` is a composite `SpacecraftComponent` (that is, both a `SpacecraftComponent` _and_ a `compositeThing`) whose `subcomponents` are its `subSystems`.

In [8]:
part def SpacecraftComponent :> MassedThing {
    attribute identifier: Scalarattributes::String;
}

part spacecraftSystem: SpacecraftComponent :> compositeThing {
    part subsystems[*] :> spacecraftSystem redefines subcomponents;    
}

PartDefinition SpacecraftComponent (4c61b269-d4d2-4ed4-9468-0ab1dd97f2df)
PartUsage spacecraftSystem (a5a4d878-7a21-4b2c-9c80-4eb24e59d87b)


## Spacecraft Decomposition

A `spacecraft` is just a top-level `spacecraftSystem` whose subsytems are the main spacecraft systems.

In [9]:
part def PropulsionSystem :> SpacecraftComponent { 
    // ...
}

part def PowerSystem :> SpacecraftComponent { 
    // ...
}

part spacecraft :> spacecraftSystem {
    part propulsion: PropulsionSystem subsets subsystems;    
    part power: PowerSystem subsets subsystems;
    // etc...
}

PartDefinition PropulsionSystem (71435e84-8ede-459b-911c-b7b140cdbf4a)
PartDefinition PowerSystem (4cff8278-0c94-4203-839a-306c79703192)
PartUsage spacecraft (a00ef966-cf88-4aac-a8ae-758f80609b7c)


## Specific Spacecraft Configuration

A configuration for a specific spacecraft gives the masses for each of the subsystems – and can even add additional subsystems as appropriate.

In [10]:
part spacecraft_1 :> spacecraft {
    import SI::*;
    
    redefines mass = 1000 [kg];
    
    part redefines propulsion {
        redefines mass = 100 [kg];
    }
    
    part redefines power {
        redefines mass = 50 [kg];
        part battery subsets subsystems {
            redefines mass = 10 [kg];
        }
    }
}

PartUsage spacecraft_1 (9d68fb69-e45f-4c39-899c-625aabd1b079)


If the `spacecraft_1::totalMass` was properly evaluated above, the result should be `1160.0 [kg]`.

# Vehicle Model Example

In [23]:
package VehicleModel_1{
    package ModelLibrary{
        package PartDefinitions{
            part def Vehicle {
                attribute mass :> ISQ::mass;
            }
            part def Engine;
            part def Cylinder;
            part def Transmission;
            part def Driveshaft;
            part def AxleAssembly;
            part def Axle;
            part def FrontAxle:>Axle{
                attribute steeringAngle:ScalarValues::Real;
            }
            part def HalfAxle;
            part def Differential;
            part def Wheel;
        }
        package PortDefinitions{
            port def FuelCmdPort;
            port def DrivePwrPort;
            port def ClutchPort;
            port def ShaftPort_a;
            port def ShaftPort_b;
            port def ShaftPort_c;
            port def ShaftPort_d;
            port def DiffPort;
            port def AxlePort;
            port def AxleToWheelPort;
            port def WheelToAxlePort;
            port def WheelToRoadPort;
            port def VehicleToRoadPort;
        }
        package FlowDefinitions{
            // This defines Torque as an alias for ISQ::Torque.
            alias Torque for ISQ::TorqueValue;
            attribute def FuelCmd;
        }
        package InterfaceDefinitions{
            import PortDefinitions::*;
            interface def EngineToTransmissionInterface{
                end p1:DrivePwrPort;
                end p2:ClutchPort;
            }
        }
        package ActionDefinitions{
            import FlowDefinitions::*;
            action def ProvidePower {
                in fuelCmd:FuelCmd;
                out wheelToRoadTorque:Torque[2];
            }
            action def GenerateTorque {
                in fuelCmd:FuelCmd;
                out engineTorque:Torque;
            }
            action def AmplifyTorque {
                in engineTorque:Torque;
                out transmissionTorque:Torque;
            }
            action def TransferTorque {
                in transmissionTorque:Torque;
                out driveshaftTorque:Torque;
            }
            action def DistributeTorque {
                in driveshaftTorque:Torque;
                out wheelToRoadTorque:Torque[2];
            }
        }
        package attributeDefinitions{
            import ScalarValues::*;
        }
    }
    package VehicleConfigurations{
        import SI::kg;
        import ModelLibrary::**;
        
        package VehicleConfiguration_a{
            package VehiclePartsTree{
                part vehicle_a:Vehicle{
                    attribute redefines mass=1750[kg];
                    part frontAxleAssembly : AxleAssembly {
                        part frontAxle:Axle;
                        part frontWheels:Wheel[2];
                    }
                    part rearAxleAssembly:AxleAssembly {
                        part rearAxle:Axle;
                        part rearWheels:Wheel[2];
                    }
                }
            }
            package VehicleActionTree{
                
            }
        }
        package VehicleConfiguration_b{
            import VehicleConfiguration_a::*;
            import VehicleConfiguration_a::VehiclePartsTree::*;
            package VehiclePartsTree{
                part vehicle_b :> vehicle_a{
                     attribute redefines mass=2000[kg];
                    /*adding ports to parts. will add ports to part defs later and redefine here*/
                    port fuelCmdPort:FuelCmdPort;
                    port vehicleToRoadPort:VehicleToRoadPort{
                        port wheelToRoadPort1:WheelToRoadPort;
                        port wheelToRoadPort2:WheelToRoadPort;
                    }
                    perform VehicleActionTree::providePower;
                    part redefines frontAxleAssembly{
                        port shaftPort_d:ShaftPort_d;
                        part frontAxle:FrontAxle redefines frontAxle;
                        part frontWheel1:>frontWheels=frontWheels#(1);
                        part frontWheel2:>frontWheels=frontWheels#(2);
                    }
                    part redefines rearAxleAssembly{
                        port shaftPort_d:ShaftPort_d;
                        perform VehicleActionTree::providePower.distributeTorque;
                        part differential:Differential{
                            port shaftPort_d:ShaftPort_d;
                            port leftDiffPort:DiffPort;
                            port rightDiffPort:DiffPort;
                        }
                        part redefines rearAxle{
                            part leftHalfAxle:HalfAxle{
                                port leftAxleToDiffPort:AxlePort;
                                port leftAxleToWheelPort:AxlePort;
                            }
                            part rightHalfAxle:HalfAxle{
                                port rightAxleToDiffPort:AxlePort;
                                port rightAxleToWheelPort:AxlePort;
                            }
                        }
                        part rearWheel1:>rearWheels=rearWheels#(1){
                            port wheelToAxlePort:WheelToAxlePort;
                            port wheelToRoadPort:WheelToRoadPort;
                        }
                        part rearWheel2:>rearWheels=rearWheels#(2){
                            port wheelToAxlePort:WheelToAxlePort;
                            port wheelToRoadPort:WheelToRoadPort;
                        }
                        bind shaftPort_d=differential.shaftPort_d;
                        connect differential.leftDiffPort to rearAxle.leftHalfAxle.leftAxleToDiffPort;
                        connect differential.rightDiffPort to rearAxle.rightHalfAxle.rightAxleToDiffPort;
                        connect rearAxle.leftHalfAxle.leftAxleToDiffPort to rearWheel1.wheelToAxlePort;
                        connect rearAxle.rightHalfAxle.rightAxleToDiffPort to rearWheel2.wheelToAxlePort;
                    }
                    part engine:Engine{
                        port fuelCmdPort:FuelCmdPort;
                        port drivePwrPort:DrivePwrPort{
                            out engineTorque:Torque;
                        }
                        part cylinders:Cylinder[4];
                        perform VehicleActionTree::providePower.generateTorque;
                    }
                    part transmission:Transmission{
                        port clutchPort:ClutchPort{
                            in engineTorque:Torque;
                        }
                        port shaftPort_a:ShaftPort_a;
                        perform VehicleActionTree::providePower.amplifyTorque;
                    }
                    part driveshaft:Driveshaft{
                        port shaftPort_b:ShaftPort_b;
                        port shaftPort_c:ShaftPort_c;
                        perform VehicleActionTree::providePower.transferTorque;
                    }
                    bind vehicle_b.fuelCmdPort=vehicle_b.engine.fuelCmdPort;
                    connect engine.drivePwrPort to transmission.clutchPort; 
                    interface engineToTransmissionInterface:EngineToTransmissionInterface
                        connect p1::>engine.drivePwrPort to p2::>transmission.clutchPort{
                             perform VehicleActionTree::providePower.generateToAmplify;
                        }
                    connect transmission.shaftPort_a to driveshaft.shaftPort_b; 
                    connect driveshaft.shaftPort_c to rearAxleAssembly.shaftPort_d;
                    bind rearAxleAssembly.rearWheel1.wheelToRoadPort=vehicle_b.vehicleToRoadPort.wheelToRoadPort1;
                    bind rearAxleAssembly.rearWheel2.wheelToRoadPort=vehicle_b.vehicleToRoadPort.wheelToRoadPort2;
                }
            }
            package VehicleActionTree{
                action providePower:ProvidePower{
                    // No successions (control flows) between these actions, because the
                    // flows between them are continuous streams.
                    action generateTorque:GenerateTorque {
                        in fuelCmd = providePower.fuelCmd;
                        out engineTorque;
                    }
                    flow generateToAmplify from generateTorque.engineTorque to amplifyTorque.engineTorque;
                    action amplifyTorque:AmplifyTorque {
                        in engineTorque;
                        out transmissionTorque;
                    }
                    flow amplifyToTransfer from amplifyTorque.transmissionTorque to transferTorque.transmissionTorque;
                    action transferTorque:TransferTorque {
                        in transmissionTorque;
                    }
                    action distributeTorque:DistributeTorque;
                }
            }
        }
    }
}

Package VehicleModel_1 (55014707-2689-4d29-88f6-3ecef98d08a2)


## "Magic" Commands

## Showing Elements

The `%show <name>` "magic" command shows the parsed syntax tree for the named element.
```sysml
part spacecraft :> spacecraftSystem {
    part propulsion: PropulsionSystem subsets subsystems;    
    part power: PowerSystem subsets subsystems;
}
```

In [25]:
%show spacecraft

PartUsage spacecraft (a00ef966-cf88-4aac-a8ae-758f80609b7c)
  [Subsetting] PartUsage spacecraftSystem (a5a4d878-7a21-4b2c-9c80-4eb24e59d87b)
  [FeatureMembership] PartUsage propulsion (51f3f492-a9f1-4398-953a-16d90f410b82)
    [FeatureTyping] PartDefinition PropulsionSystem (71435e84-8ede-459b-911c-b7b140cdbf4a)
    [Subsetting] PartUsage subsystems (1597d02c-da90-4741-a551-ebff5362d496)
  [FeatureMembership] PartUsage power (eea96d6c-fb8d-4d7f-be80-d2200b82d87d)
    [FeatureTyping] PartDefinition PowerSystem (4cff8278-0c94-4203-839a-306c79703192)
    [Subsetting] PartUsage subsystems (1597d02c-da90-4741-a551-ebff5362d496)


## Publishing Models

The `%publish <name>` command "publishes" the model graph rooted in the named element to a prototype repository via the proposed standard API. 

* The API endpoint currently used is for a single, publicly available instance of the repository.
* All elements directly *and* indirectly accessible from the specified root element are saved (but not indirectly referenced standard library elements).
* Each publish of a model creates a new copy in the repository.